In [13]:
import collections
import json
import os
import re
from dataclasses import dataclass, field
from typing import NamedTuple

import pandas as pd
import torch
from colorama import Fore, Style
from fuzzywuzzy import fuzz
from transformers import TrainingArguments
from tqdm import tqdm

In [14]:
# Copied from train.py
@dataclass
class LeiaTrainingArguments(TrainingArguments):
    model_name_or_path: str | None = field(default=None)
    use_flash_attention_2: bool = field(default=False)

    wikipedia_dataset_dir: str | None = field(default=None)

    trans_insertion_strategy: str = field(default="none")
    trans_insertion_prob: float = field(default=1.0)
    trans_insertion_prob_decay: bool = field(default=False)
    trans_insertion_min_prob: float = field(default=0.0)

    max_length: int = field(default=1024)

    eval_tasks: str | None = field(default=None)
    max_eval_samples_for_tasks: int | None = field(default=None)
    num_fewshot_samples_for_tasks: str | None = field(default=None)
    use_dynamic_generation_length: bool = field(default=True)


class Task(NamedTuple):
    name: str
    language: str


class Result(NamedTuple):
    task: Task
    metric: str
    value: float


class Experiment(NamedTuple):
    name: str
    args: LeiaTrainingArguments | None
    results: list[Result]
    predictions: dict[Task, list[dict]]

In [15]:
TASK2LANGUAGE = {
    "jsquad": "ja",
    "jemhopqa": "ja",
    "jaqket": "ja",
    "jcommonsenseqa": "ja",
    "niilc": "ja",
}


def parse_file_name(file_name: str) -> Task | None:
    if file_name.endswith("_metrics.json"):
        task_name = file_name[: -len("_metrics.json")]
    elif file_name.endswith("_predictions.jsonl"):
        task_name = file_name[: -len("_predictions.jsonl")]

    match_obj = re.match(r"^([a-z_]+)_([a-z]{2})$", task_name)
    if match_obj:
        task_name, language = match_obj.group(1), match_obj.group(2)
        return Task(task_name, language)
    elif task_name in TASK2LANGUAGE:
        return Task(task_name, TASK2LANGUAGE[task_name])

    print(f"Could not parse task name: {task_name}")
    return None

In [16]:
def load_experiments(runs_dir: str) -> dict[str, Experiment]:
    experiments: dict[str, Experiment] = {}
    for run_dir in os.listdir(runs_dir):
        if not os.path.isdir(os.path.join(runs_dir, run_dir, "results")):
            tqdm.write(f"results directory not found. Skipping {run_dir}")
            continue

        args = None
        if os.path.exists(os.path.join(runs_dir, run_dir, "training_args.bin")):
            args = torch.load(os.path.join(runs_dir, run_dir, "training_args.bin"))

        results = []
        predictions = {}
        for result_file in os.listdir(os.path.join(runs_dir, run_dir, "results")):
            if result_file.endswith("_metrics.json"):
                with open(os.path.join(runs_dir, run_dir, "results", result_file)) as f:
                    task = parse_file_name(result_file)
                    if task is not None:
                        for metric, value in json.load(f).items():
                            results.append(Result(task, metric, value))

            elif result_file.endswith("_predictions.jsonl"):
                task = parse_file_name(result_file)
                if task is not None:
                    with open(os.path.join(runs_dir, run_dir, "results", result_file)) as f:
                        predictions[task] = [json.loads(line) for line in f]

        experiments[run_dir] = Experiment(run_dir, args, results, predictions)

    return experiments

## Experiments based on LLaMA 2 7B


### Results


In [17]:
experiments = load_experiments("runs")

target_base_models = ["Llama-2-7b-hf"]
target_tasks = {"xcsqa", "xcodah", "xnli"}
conditions = {
    "learning_rate": {5e-6},
    "max_steps": {20, 50},
}
leia_conditions = {
    "learning_rate": {5e-6},
    "max_steps": {20, 50},
    "trans_insertion_prob": {0.5, 1.0},
}

data = collections.defaultdict(lambda: collections.defaultdict(dict))
task2languages = collections.defaultdict(set)
for experiment_name, experiment in experiments.items():
    if experiment.args is None:  # base model
        if experiment_name in target_base_models:
            for result in experiment.results:
                data[experiment_name][result.task.name][result.task.language] = result.value
        continue

    elif experiment.args.trans_insertion_strategy == "none" and all(
        getattr(experiment.args, k) in v for k, v in conditions.items()
    ):
        for result in experiment.results:
            data["leia_none"][result.task.name][result.task.language] = result.value
            task2languages[result.task.name].add(result.task.language)

    elif experiment.args.trans_insertion_strategy in ("left", "right", "replace") and all(
        getattr(experiment.args, k) in v for k, v in leia_conditions.items()
    ):
        for result in experiment.results:
            row_name = f"leia_{experiment.args.trans_insertion_strategy}_{experiment.args.trans_insertion_prob}"
            data[row_name][result.task.name][result.task.language] = result.value
            task2languages[result.task.name].add(result.task.language)

for task, languages in task2languages.items():
    if task not in target_tasks:
        continue
    rows = []
    languages = sorted(languages)
    row_names = []
    for model_name, results in sorted(data.items(), key=lambda x: x[0]):
        row = [results.get(task, {}).get(language) for language in languages]
        rows.append(row)
        row_names.append(model_name)
    rows.append(
        [
            data["leia_right_0.5"].get(task, {}).get(language, 0.0) - data["leia_none"].get(task, {}).get(language, 0.0)
            for language in languages
        ]
    )
    rows.append(
        [
            data["leia_right_0.5"].get(task, {}).get(language, 0.0)
            - data["Llama-2-7b-hf"].get(task, {}).get(language, 0.0)
            for language in languages
        ]
    )
    column_names = languages + ["avg"]
    for row in rows:
        row.append(sum([x for x in row if x is not None]) / len(row))
    row_names = row_names + ["right_0.5 - none", "right_0.5 - llama2"]
    df = pd.DataFrame(rows, columns=column_names, index=row_names)
    df = df.style.highlight_max(color="gray", axis=0)
    display(f"{task}:")
    display(df)

'xcsqa:'

,ar,es,hi,ja,ru,sw,avg
Llama-2-7b-hf,0.210000,0.456000,0.196000,0.357000,0.354000,0.168000,0.290167
leia_left_0.5,0.213000,0.469000,0.198000,0.355000,0.362000,0.177000,0.295667
leia_left_1.0,0.210000,0.465000,0.201000,0.351000,0.359000,0.175000,0.293500
leia_none,0.208000,0.457000,0.202000,0.349000,0.356000,0.168000,0.290000
leia_replace_0.5,0.212000,0.465000,0.199000,0.354000,0.359000,0.179000,0.294667
leia_replace_1.0,0.208000,0.464000,0.199000,0.351000,0.357000,0.174000,0.292167
leia_right_0.5,0.212000,0.468000,0.204000,0.355000,0.362000,0.178000,0.296500
leia_right_1.0,0.207000,0.467000,0.200000,0.353000,0.360000,0.177000,0.294000
right_0.5 - none,0.004000,0.011000,0.002000,0.006000,0.006000,0.010000,0.006500
right_0.5 - llama2,0.002000,0.012000,0.008000,-0.002000,0.008000,0.010000,0.006333


'xcodah:'

,ar,es,hi,ja,ru,sw,avg
Llama-2-7b-hf,0.306667,0.446667,0.296667,0.300000,0.346667,0.286667,0.330556
leia_left_0.5,0.330000,0.460000,0.303333,0.343333,0.376667,0.300000,0.352222
leia_left_1.0,0.323333,0.450000,0.303333,0.333333,0.380000,0.300000,0.348333
leia_none,0.303333,0.446667,0.266667,0.303333,0.343333,0.290000,0.325556
leia_replace_0.5,0.330000,0.456667,0.303333,0.346667,0.366667,0.303333,0.351111
leia_replace_1.0,0.323333,0.450000,0.300000,0.340000,0.380000,0.303333,0.349444
leia_right_0.5,0.336667,0.460000,0.303333,0.343333,0.373333,0.303333,0.353333
leia_right_1.0,0.323333,0.450000,0.303333,0.340000,0.376667,0.300000,0.348889
right_0.5 - none,0.033333,0.013333,0.036667,0.040000,0.030000,0.013333,0.027778
right_0.5 - llama2,0.030000,0.013333,0.006667,0.043333,0.026667,0.016667,0.022778


'xnli:'

,ar,es,hi,ru,sw,avg
Llama-2-7b-hf,0.347904,0.420160,0.382834,0.428743,0.348104,0.385549
leia_left_0.5,0.344112,0.426946,0.391617,0.437924,0.340519,0.388224
leia_left_1.0,0.344910,0.422555,0.400000,0.439321,0.341317,0.389621
leia_none,0.347505,0.427146,0.368663,0.418164,0.338124,0.379920
leia_replace_0.5,0.345908,0.423952,0.390220,0.437525,0.340719,0.387665
leia_replace_1.0,0.344311,0.423952,0.395210,0.436727,0.342116,0.388463
leia_right_0.5,0.344711,0.426148,0.392016,0.437126,0.340319,0.388064
leia_right_1.0,0.344910,0.425948,0.397206,0.440319,0.340918,0.389860
right_0.5 - none,-0.002794,-0.000998,0.023353,0.018962,0.002196,0.008144
right_0.5 - llama2,-0.003194,0.005988,0.009182,0.008383,-0.007784,0.002515


### Analysis


In [18]:
for item1, item2 in zip(
    experiments["ja_llama2_none_lr5e-6_step50"].predictions[Task("xcodah", "ja")],
    experiments["ja_llama2_right_p0.5_lr5e-6_step50"].predictions[Task("xcodah", "ja")],
):
    assert item1["example"]["id"] == item2["example"]["id"]

    if item1["prediction"] != item2["prediction"]:
        label = ["A", "B", "C", "D"].index(item1["example"]["answerKey"])
        # print(item1['example']['question']['choices']['text'])
        print(Fore.GREEN if item1["prediction"] == label else Fore.RED, end="")
        print("LEIAなし:", item1["example"]["question"]["choices"]["text"][item1["prediction"]])
        print(Fore.GREEN if item2["prediction"] == label else Fore.RED, end="")
        print("LEIAあり:", item1["example"]["question"]["choices"]["text"][item2["prediction"]])
        print(Style.RESET_ALL, end="")
        print("正解:", item1["example"]["question"]["choices"]["text"][label])
        print("---")

LEIAなし: サッカーの試合が行われています。 サッカー選手がチームメイトにボールを渡す
LEIAあり: サッカーの試合が行われています。 フィールドで練習するサッカー選手
正解: サッカーの試合が行われています。 サッカー選手がチームメイトにボールを渡す
---
LEIAなし: 曲芸師は、ポリアモリーが彼らのためのものではないことを知っていた。 曲芸師は3人の恋人を扱えなくなった。
LEIAあり: 曲芸師は、ポリアモリーが彼らのためのものではないことを知っていた。 ジャグラーもアクロバットでした。
正解: 曲芸師は、ポリアモリーが彼らのためのものではないことを知っていた。 曲芸師は3人の恋人を扱えなくなった。
---
LEIAなし: 海を沸かせようとしている。 海が大好きなんです。
LEIAあり: 海を沸かせようとしている。 彼は不可能なことを成し遂げようとしている。
正解: 海を沸かせようとしている。 彼は不可能なことを成し遂げようとしている。
---
LEIAなし: 彼は羊のようです。 怠け者です。
LEIAあり: 彼は羊のようです。 彼は面白い。
正解: 彼は羊のようです。 彼は騙されやすい。
---
LEIAなし: スナイパーはスコープを覗き込む。 スナイパーが本をスナイプする。
LEIAあり: スナイパーはスコープを覗き込む。 狙撃手がメイドを撃つ。
正解: スナイパーはスコープを覗き込む。 狙撃手は武将を狙う。
---
LEIAなし: 男の子がコンピューターゲームをしています。 彼は男の子です
LEIAあり: 男の子がコンピューターゲームをしています。 彼は疲れている
正解: 男の子がコンピューターゲームをしています。 彼はやめたくないんだ
---
LEIAなし: タイルの上を蟻が行進しています。 踏まれたがっている。
LEIAあり: タイルの上を蟻が行進しています。 家まで食べ物を運んでいる。
正解: タイルの上を蟻が行進しています。 家まで食べ物を運んでいる。
---
LEIAなし: イギリスのロンドンにあるエッフェル塔。
LEIAあり: エッフェル塔はパリにあります。
正解: エッフェル塔はパリにあります。
---
LEIAなし: ウィルソンは頭が良く、数学を勉強するのが大好きな男の子です。 ウィルソンは早起き
LEIAあり: ウィルソンは頭が

In [19]:
for item1, item2 in zip(
    experiments["ja_llama2_none_lr5e-6_step50"].predictions[Task("xcsqa", "ja")],
    experiments["ja_llama2_right_p0.5_lr5e-6_step50"].predictions[Task("xcsqa", "ja")],
):
    assert item1["example"]["id"] == item2["example"]["id"]
    if item1["prediction"] != item2["prediction"]:
        label = ["A", "B", "C", "D", "E"].index(item1["example"]["answerKey"])
        # print(item1['example']['question']['choices']['text'])
        print(Style.RESET_ALL, end="")
        print("問題文:", item1["example"]["question"]["stem"])
        print(Fore.GREEN if item1["prediction"] == label else Fore.RED, end="")
        print(
            "LEIAなし:",
            item1["example"]["question"]["choices"]["text"][item1["prediction"]],
        )
        print(Fore.GREEN if item2["prediction"] == label else Fore.RED, end="")
        print(
            "LEIAあり:",
            item2["example"]["question"]["choices"]["text"][item2["prediction"]],
        )
        print("正解:", item2["example"]["question"]["choices"]["text"][label])
        print("---")

問題文: テニスコートでミトン型になっているとしたら、あなたはどの状態ですか？
LEIAなし: タウン
LEIAあり: カントリークラブ
正解: ミシガン
---
問題文: 並んで何を達成するのか？
LEIAなし: 怒り
LEIAあり: 疲労
正解: 順番待ち
---
問題文: 思想信条が軽度ではなく極端な人は何なのか？
LEIAなし: 大事
LEIAあり: 急進的
正解: 急進的
---
問題文: ピエモンテのシャツ工場に綿を送る必要があったのですが、どこに送ればいいのでしょうか？
LEIAなし: 織物
LEIAあり: ジョージア
正解: ノースカロライナ
---
問題文: あまりにも良いと思われる製品の有効性の証拠を提示されたとき、経験豊富な消費者は何をすべきでしょうか？
LEIAなし: 読む
LEIAあり: 吟味
正解: 吟味
---
問題文: ジョンはなぜそれがブローと呼ばれているのか知らなかった。吹いていなかった。 ブローイングはなかった。 吹いたのではなく、何をされたのでしょうか？
LEIAなし: 吸い込む
LEIAあり: 吸い込み
正解: しゃぶる
---
問題文: なぜ人は薬を飲むのか？
LEIAなし: 儲ける
LEIAあり: 褒め言葉
正解: 痛くも痒くもない
---
問題文: 彼女は布を媒体として楽しんでいましたが、彼女の創造性は、彼女が何をするときに本当に示したのでしょうか？
LEIAなし: 縫う
LEIAあり: 書く
正解: 縫う
---
問題文: アルゼンチンのクリスマスはいつ？
LEIAなし: 夏
LEIAあり: イースター
正解: 夏
---
問題文: 接着剤がよく使われている場所は？
LEIAなし: 派
LEIAあり: キャビネット
正解: 美術室
---
問題文: 定期的に脱走する犬にありそうなこととは？
LEIAなし: 忠実
LEIAあり: 疲れた
正解: とっても頭がいい
---
問題文: クウェートなどはすぐには連想されませんが、どこに位置しているのでしょうか？
LEIAなし: アジア
LEIAあり: アラビア
正解: アジア
---
問題文: お酒を飲むと何につながるのか？
LEIAなし: 酩酊
LEIAあり: 嘔吐
正解: 酩酊
---
問題文: 彼はポーカーをプレイするためにテーブルに座って、彼は何に持っていた手を結合す

## Experiments based on Swallow-7B


### Results


In [20]:
experiments = load_experiments("runs_swallow")

target_base_models = ["Swallow-7b-hf"]

data = collections.defaultdict(lambda: collections.defaultdict(dict))
result_names = set()
for experiment_name, experiment in experiments.items():
    if experiment.args is None:  # base model
        if experiment_name in target_base_models:
            for result in experiment.results:
                result_name = f"{result.task.name}_{result.metric}"
                data[experiment_name][result_name] = result.value
                result_names.add(result_name)

    elif experiment.args.trans_insertion_strategy == "none":
        for result in experiment.results:
            result_name = f"{result.task.name}_{result.metric}"
            data["leia_none"][result_name] = result.value
            result_names.add(result_name)

    elif experiment.args.trans_insertion_strategy in ("left", "right", "replace"):
        for result in experiment.results:
            result_name = f"{result.task.name}_{result.metric}"
            row_name = f"leia_{experiment.args.trans_insertion_strategy}_{experiment.args.trans_insertion_prob}"
            data[row_name][result_name] = result.value
            result_names.add(result_name)

result_names = sorted(result_names)
rows = []
row_names = []
for model_name, results in sorted(data.items(), key=lambda x: x[0]):
    row = [results.get(result_name) for result_name in result_names]
    rows.append(row)
    row_names.append(model_name)

for row in rows:
    row.append(sum([x for x in row if x is not None]) / len(row))

column_names = result_names + ["avg"]
df = pd.DataFrame(rows, columns=column_names, index=row_names)
df = df.style.highlight_max(color="gray", axis=0)
display(df)

,jaqket_accuracy,jcommonsenseqa_accuracy,jemhopqa_char_f1,jsquad_exact_match,jsquad_f1,niilc_char_f1,xcodah_accuracy,xcsqa_accuracy,xwinograd_accuracy,avg
Swallow-7b-hf,0.955720,0.798928,0.507833,0.815398,0.900608,0.595101,0.420000,0.410000,0.786236,0.687758
leia_none,0.955720,0.795353,0.501417,0.811571,0.900144,0.577576,0.410000,0.396000,0.773723,0.680167
leia_right_0.5,0.963100,0.800715,0.545000,0.813372,0.900369,0.601667,0.423333,0.422000,0.793535,0.695899


### Analysis


In [21]:
for item1, item2 in zip(
    experiments["swallow_none_lr5e-6_step50"].predictions[Task("niilc", "ja")],
    experiments["swallow_right_p0.5_lr5e-6_step50"].predictions[Task("niilc", "ja")],
):
    if item1["prediction"] != item2["prediction"]:
        score1 = fuzz.token_sort_ratio(item1["prediction"], item1["example"]["output"]) / 100.0
        score2 = fuzz.token_sort_ratio(item2["prediction"], item1["example"]["output"]) / 100.0
        print(Style.RESET_ALL, end="")
        print("問題文:", item1["example"]["input"])

        print(Fore.GREEN if score1 > score2 else Fore.RED, end="")
        print("LEIAなし:", item1["prediction"])
        print(Fore.GREEN if score1 <= score2 else Fore.RED, end="")
        print("LEIAあり:", item2["prediction"])
        print(Style.RESET_ALL, end="")
        print("正解:", item1["example"]["output"])
        print("---")

問題文: 質問：石油輸出国機構の本部はどこにある？
LEIAなし: サウジアラビアのリヤド
LEIAあり: ウィーン
正解: ウィーン
---
問題文: 質問：パリに本部のある国連の機関の名前を教えて下さい。
LEIAなし: 国際連合教育科学文化機関
LEIAあり: 国際連合教育科学文化機関(ユネスコ)
正解: ユネスコ,UNESCO,国際連合教育科学文化機関
---
問題文: 質問：大韓民国と朝鮮民主主義人民共和国を隔てている緯度は？
LEIAなし: 北緯38度
LEIAあり: 北緯38度線
正解: 北緯38度
---
問題文: 質問：日本の観測史上最低の気温は何度か。
LEIAなし: -27.2°C
LEIAあり: -27.2°C(1902年1月25日 北海道旭川市)
正解: マイナス41度
---
問題文: 質問：四鏡とは具体的に何を指すか？
LEIAなし: 『発心集』『古今著聞集』『古事談』『宇治拾遺物語』の4つの説話集の総称
LEIAあり: 『発心集』『古今著聞集』『宇治拾遺物語』『今昔物語集』の4つの説話集の総称
正解: 大鏡,今鏡,水鏡,増鏡
---
問題文: 質問：ガラスはどの化学物質から出来ているの？
LEIAなし: 二酸化ケイ素
LEIAあり: 二酸化ケイ素(SiO2)
正解: ケイ酸塩
---
問題文: 質問：日本で二番目に大きな湖は？
LEIAなし: 霞ヶ浦
LEIAあり: 琵琶湖
正解: 霞ヶ浦
---
問題文: 質問：国の宗教がイスラム教なのはどこの国か。
LEIAなし: サウジアラビア,イラン,インドネシア,マレーシア,パキスタン,バングラデシュ,モルディブ,アラブ首長国連邦,エジプト,トルコ,アルジェリア,モロッコ,チュニジア,スーダン,イエメン,アフガニスタン,イラク,シリア,レバノン,クウェート,オマーン,バーレーン,カタール,ヨルダン,パレスチナ,イエメン,ソマリア,モルディブ,アフガニスタン,イラク,シリア,レバノン,クウェート,オマーン,バーレーン,カタール,ヨルダン,パレスチナ,イエメン,ソマリア
LEIAあり: サウジアラビア,イラン,インドネシア,マレーシア,パキスタン,バングラデシュ,モルディブ,アラブ首長国連邦,エジプト,トルコ,アルジェリア,モロッコ,チュニジア,スーダン,イエメン,アフガニスタン,イラク,シリア

In [22]:
for item1, item2 in zip(
    experiments["swallow_none_lr5e-6_step50"].predictions[Task("jemhopqa", "ja")],
    experiments["swallow_right_p0.5_lr5e-6_step50"].predictions[Task("jemhopqa", "ja")],
):
    if item1["prediction"] != item2["prediction"]:
        score1 = fuzz.token_sort_ratio(item1["prediction"], item1["example"]["output"]) / 100.0
        score2 = fuzz.token_sort_ratio(item2["prediction"], item1["example"]["output"]) / 100.0
        print(Style.RESET_ALL, end="")
        print("問題文:", item1["example"]["input"])

        print(Fore.GREEN if score1 > score2 else Fore.RED, end="")
        print("LEIAなし:", item1["prediction"])
        print(Fore.GREEN if score1 <= score2 else Fore.RED, end="")
        print("LEIAあり:", item2["prediction"])
        print(Style.RESET_ALL, end="")
        print("正解:", item1["example"]["output"])
        print("---")

問題文: 質問：藤原寛の出身校を設置した学校法人の名称は？
LEIAなし: 学校法人東京農業大学
LEIAあり: 学校法人 東京農業大学
正解: 学校法人関西学院
---
問題文: 質問：『仮面ライダーW』『仮面ライダー龍騎』、先に放送開始されたのは『仮面ライダーW』ですか？
LEIAなし: YES
LEIAあり: NO
正解: NO
---
問題文: 質問：犬飼貴丈が所属する事務所の代表者は誰ですか？
LEIAなし: 犬飼貴丈
LEIAあり: バーニングプロダクション
正解: 周防郁雄
---
問題文: 質問：徳川家宣の父方の祖父の死没年月日は西暦でいつですか？
LEIAなし: 1649年1月1日
LEIAあり: 1651年1月2日
正解: 1651年6月8日
---
問題文: 質問：佐々木主浩が所属する団体の現組織の設立年月日はいつですか？
LEIAなし: 1999年12月1日
LEIAあり: 1998年12月1日
正解: 40466
---
問題文: 質問：映画『風の谷のナウシカ』のプロデューサーが手掛けた、赤毛の少女を主人公とする作品のタイトルは何でしょうか？
LEIAなし: 『赤毛のアン』
LEIAあり: 『耳をすませば』
正解: 赤毛のアン
---
問題文: 質問：YOSHIKIとPATA、誕生日が早いのはYOSHIKIですか？
LEIAなし: YES
LEIAあり: NO
正解: NO
---
問題文: 質問：豊岡市と伊丹市の管轄区域は、どちらも但馬県民局ですか？
LEIAなし: YES
LEIAあり: NO
正解: NO
---
問題文: 質問：『男はつらいよ』の主演俳優の誕生日は何月何日？
LEIAなし: 1月1日
LEIAあり: 1937年1月1日
正解: 3月10日
---
問題文: 質問：映画『コクリコ坂』からを配給した会社が入っているビルの名前は何ですか？
LEIAなし: TOHOシネマズ六本木ヒルズ
LEIAあり: 東京都港区六本木6丁目10番1号 六本木ヒル
正解: 東宝日比谷ビル
---
問題文: 質問：IPodを製作している企業の本社所在地は？
LEIAなし: アメリカ合衆国カリフォルニア州
LEIAあり: アメリカ合衆国カリフォルニア州クパチーノ
正解: カリフォルニア州クパチーノ
---
問題文: 質問：香取神宮と日光二荒山神社

In [23]:
for item1, item2 in zip(
    experiments["swallow_none_lr5e-6_step50"].predictions[Task("jcommonsenseqa", "ja")],
    experiments["swallow_right_p0.5_lr5e-6_step50"].predictions[Task("jcommonsenseqa", "ja")],
):
    if item1["prediction"] != item2["prediction"]:
        print(Style.RESET_ALL, end="")
        print("問題文:", item1["example"]["question"])

        print(Fore.GREEN if item1["prediction"] == item1["example"]["label"] else Fore.RED, end="")
        print("LEIAなし:", item1["example"][f"choice{item1['prediction']}"])
        print(Fore.GREEN if item2["prediction"] == item1["example"]["label"] else Fore.RED, end="")
        print("LEIAあり:", item1["example"][f"choice{item2['prediction']}"])
        print(Style.RESET_ALL, end="")
        print("正解:", item1["example"][f"choice{item1['example']['label']}"])
        print("---")

問題文: 魚の名前がついているのは？
LEIAなし: かかと
LEIAあり: ヒラメ筋
正解: ヒラメ筋
---
問題文: しっぽのけものはどれ？
LEIAなし: 害獣
LEIAあり: 獣医学
正解: 尾獣
---
問題文: 家の中にあるのは？
LEIAなし: 地下道
LEIAあり: 板
正解: 居間
---
問題文: 土と氷が混ざって固まっているものは？
LEIAなし: 凍土
LEIAあり: 粘土
正解: 凍土
---
問題文: お節介で手伝ったり等あまり良い意味で使われない行為を何と言う？
LEIAなし: あげる
LEIAあり: 盗む
正解: 手を出す
---
問題文: 仕事をしているうえで上役の人を何という？
LEIAなし: 上司
LEIAあり: 取締役
正解: 上司
---
問題文: ブラウン管や液晶、プラズマなどの家電は？
LEIAなし: テレビ
LEIAあり: エアコン
正解: テレビ
---
問題文: この3つの中で2つの車輪を表す名称は？
LEIAなし: ゴミ
LEIAあり: 二輪
正解: 二輪
---
問題文: 行政区画は？
LEIAなし: 郡
LEIAあり: 町内
正解: 郡
---
問題文: キオスクなどの形態を何という？
LEIAなし: 商店
LEIAあり: 駅の売店
正解: 駅の売店
---
問題文: 音楽的な高低・調子などをつけて発声することを何と言うか？
LEIAなし: 演奏する
LEIAあり: 歌う
正解: 歌う
---
問題文: 日本と親しくすることは？
LEIAなし: 訪日
LEIAあり: 親日
正解: 親日
---
問題文: 桜の仲間で夏から秋に実をつける果物は？
LEIAなし: リンゴ
LEIAあり: みかん
正解: リンゴ
---
問題文: いつも元気で笑顔の人は？
LEIAなし: あたたかい
LEIAあり: 明るい
正解: 明るい
---
問題文: 人それぞれなのは？
LEIAなし: オルガン
LEIAあり: 人物像
正解: 人物像
---
問題文: 文字だけでは伝わりにくい時に伝わりやすくしたものは？
LEIAなし: 写真入り
LEIAあり: 天体写真
正解: 写真入り
---
問題文: この中で、しきたりはどれ？
LEIAなし: フロリダ州
LEIAあり: 葬儀
正解: 葬儀
---
問題文: 発声するとどうなる？
L

In [24]:
for item1, item2 in zip(
    experiments["swallow_none_lr5e-6_step50"].predictions[Task("jaqket", "ja")],
    experiments["swallow_right_p0.5_lr5e-6_step50"].predictions[Task("jaqket", "ja")],
):
    if item1["prediction"] != item2["prediction"]:
        print(Style.RESET_ALL, end="")
        print("問題文:", item1["example"]["question"])

        print(Fore.GREEN if item1["prediction"] == item1["example"]["label"] else Fore.RED, end="")
        print("LEIAなし:", item1["example"]["answer_candidates"][item1["prediction"]])
        print(Fore.GREEN if item2["prediction"] == item1["example"]["label"] else Fore.RED, end="")
        print("LEIAあり:", item1["example"]["answer_candidates"][item2["prediction"]])
        print(Style.RESET_ALL, end="")
        print("正解:", item1["example"]["answer_candidates"][item1["example"]["label"]])
        print("---")

問題文: アメリカ・メリーランド州の州都で、海軍士官学校の所在地として知られるのはどこ?
LEIAなし: フィラデルフィア
LEIAあり: アナポリス (メリーランド州)
正解: アナポリス (メリーランド州)
---
問題文: 地井明子によってデザインされた、オスのゴールデン・レトリバーであるプリンを主人公とするサンリオのキャラクター群は何でしょう?
LEIAなし: ポムポムプリン
LEIAあり: シナモロール
正解: ポムポムプリン
---
問題文: 2010年にリリースされたいきものがかりの曲で、NHK連続テレビ小説『ゲゲゲの女房』の主題歌であったのは何?
LEIAなし: 逢いたくていま
LEIAあり: ありがとう (いきものがかりの曲)
正解: ありがとう (いきものがかりの曲)
---
問題文: 世界にある「白い山」という意味の山のうち、標高が最も高い、8167mの世界第7位の高峰は何?
LEIAなし: エベレスト
LEIAあり: チョ・オユー
正解: ダウラギリ
---
問題文: バンド名は「弱者の反撃」という意味で名づけられた、『天体観測』や『オンリー ロンリー グローリー』などの代表曲がある音楽グループは何でしょう?
LEIAなし: いきものがかり
LEIAあり: BUMP OF CHICKEN
正解: BUMP OF CHICKEN
---
問題文: 見える時間によって「明けの明星」「宵の明星」と呼ばれる、太陽系の第2惑星といえば何でしょう?
LEIAなし: 木星
LEIAあり: 金星
正解: 金星
---
問題文: 初代大統領が「バナナさん」だったアフリカの国はどこ?
LEIAなし: ナミビア
LEIAあり: ブルンジ
正解: ジンバブエ
---
問題文: 摩擦により温度が60°C以上になると、文字が消える特殊なインクが使われている、パイロットが開発した筆記具は何でしょう?
LEIAなし: フリクション (筆記具)
LEIAあり: SARASA
正解: フリクション (筆記具)
---
